(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Recommender System for Amazon Electronics

In this assignment, we will be working with the [Amazon dataset](http://cs-people.bu.edu/kzhao/teaching/amazon_reviews_Electronics.tar.gz). You will build a recommender system to make predictions related to reviews of Electronics products on Amazon.

Your grades will be determined by your performance on the predictive tasks as well as a brief written report about the approaches you took.

This assignment should be completed **individually**.

## Files

**train.json** 1,000,000 reviews to be used for training. It is not necessary to use all reviews for training if doing so proves too computationally intensive. The fields in this file are:

* **reviewerID** The ID of the reviewer. This is a hashed user identifier from Amazon.

* **asin** The ID of the item. This is a hashed product identifier from Amazon.

* **overall** The rating of reviewer gave the item.

* **helpful** The helpfulness votes for the review. This has 2 subfields, 'nHelpful' and 'outOf'. The latter is the total number of votes this review received. The former is the number of those that considered the review to be helpful.

* **reviewText** The text of the review.

* **summary** The summary of the review.

* **unixReviewTime** The time of the review in seconds since 1970.

**meta.json** Contains metadata of the items:

* **asin** The ID of the item.

* **categories** The category labels of the item being reviewed.

* **price** The price of the item.

* **brand** The brand of the item.

**pairs_Rating.txt** The pairs (reviewerID and asin) on which you are to predict ratings.

**pairs_Purchase.txt** The pairs on which you are to predict whether a user purchased an item or not.

**pairs_Helpful.txt** The pairs on which you are to predict helpfulness votes. A third column in this file is the total number of votes from which you should predict how many were helpful.

**helpful.json** The review data associated with the helpfulness prediction test set. The 'nHelpful' field has been removed from this data since that is the value you need to predict above. This data will only be of use for the helpfulness prediction task.

**baseline.py** A simple baseline for each task.

## Tasks

**Rating prediction** Predict people's star ratings as accurately as possible for those (reviewerID, asin) pairs in 'pairs_Rating.txt'. Accuracy will be measured in terms of the [root mean-squared error (RMSE)](http://www.kaggle.com/wiki/RootMeanSquaredError).

**Purchase prediction** Predict given a (reviewerID, asin) pair from 'pairs_Purchase.txt' whether the user purchased the item (really, whether it was one of the items they reviewed). Accuracy will be measured in terms of the [categorization accuracy](http://www.kaggle.com/wiki/HammingLoss) (1 minus the Hamming loss).

**Helpfulness prediction** Predic whether a user's review of an item will be considered helpful. The file 'pairs_Helpful.txt' contains (reviewerID, asin) pairs with a third column containing the number of votes the user's review of the item received. You must predict how many of them were helpful. Accuracy will be measured in terms of the total [absolute error](http://www.kaggle.com/wiki/AbsoluteError), i.e. you are penalized one according to the difference |nHelpful - prediction|, where 'nHelpful' is the number of helpful votes the review actually received, and 'prediction' is your prediction of this quantity.

We set up competitions on Kaggle to keep track of your results compared to those of other members of the class. The leaderboard will show your results on half of the test data, but your ultimate score will depend on your predictions across the whole dataset.
* Kaggle competition: [rating prediction](https://inclass.kaggle.com/c/cs591-hw3-rating-prediction3) click here to [join](https://kaggle.com/join/datascience16rating)
* Kaggle competition: [purchase prediction](https://inclass.kaggle.com/c/cs591-hw3-purchase-prediction) click here to [join](https://kaggle.com/join/datascience16purchase)
* Kaggle competition: [helpfulness prediction](https://inclass.kaggle.com/c/cs591-hw3-helpful-prediction) click here to [join](https://kaggle.com/join/datascience16helpful)

## Grading and Evaluation

You will be graded on the following aspects.

* Your written report. This should describe the approaches you took to each of the 3 tasks. To obtain good performance, you should not need to invent new approaches (though you are more than welcome to) but rather you will be graded based on your decision to apply reasonable approaches to each of the given tasks. (**10pts** for each task)

* Your ability to obtain a solution which outperforms the baselines on the unseen portion of the test data. Obtaining full marks requires a solution which is substantially better (at least several percent) than baseline performance. (**10pts** for each task)

* Your ranking for each of the three tasks compared to other students in the class. (**5pts** for each task)

* Obtain a solution which outperforms the baselines on the seen portion of the test data (the leaderboard). 
(**5pts** for each task)

## Baselines

Simple baselines have been provided for each of the 3 tasks. These are included in 'baselines.py' among the files above. These 3 baselines operate as follows:

**Rating prediction** Returns the global average rating, or the user's average if you have seen them before in the training data.

**Purchase prediction** Finds the most popular products that account for 50% of purchases in the training data. Return '1' whenever such a product is seen at test time, '0' otherwise.

** Helpfulness prediction** Multiplies the number of votes by the global average helpfulness rate, or the user's rate if we saw this user in the training data.

Running 'baseline.py' produces 3 files containing predicted outputs. Your submission files should have the same format.

## Dataset Citation

**Image-based recommendations on styles and substitutes** J. McAuley, C. Targett, J. Shi, A. van den Hengel *SIGIR*, 2015

**Inferring networks of substitutable and complementary products** J. McAuley, R. Pandey, J. Leskovec *Knowledge Discovery and Data Mining*, 2015

-----------------

In [57]:
# Parse data and feature extract, pickle resulting dictionaries for easier loading

from textblob import TextBlob

def build_item_data():
    meta = 'amazon_reviews_Electronics/meta.json'
    items = {}
    item = None
    cats = None
    price = None
    for line in open(meta):
        item = eval(line)
        cats = eval(item['categories'].replace('"', '')\
                    .replace("',", '",').replace(" '", ' "')\
                    .replace("']", '"]').replace("['", '["'))
        try:
            price = float(item['price'])
        except:
            price = None
        items[item['asin']] = {'category': [x[0] for x in cats],
                               'price': price,
                               'review_num': 0,
                               'avg_polarity': [],
                               'avg_subjectivity': [],
                               'avg_rating': []}           
    return items

def build_user_data(items):
    
    def new_user():
        return {'item': [],
                'polarity': [],
                'subjectivity': [],
                'rating': [],
                'text': [],
                'helpful_vote': [],
                'total_vote': [],
                'category': {},
                'stats': {}}
    
    users = {}
    for line in open('amazon_reviews_Electronics/train.json'):
        review = eval(line)
        user = review['reviewerID']
        item = review['asin']
        categories = items[item]['category']
        polarity, subjectivity = TextBlob(review['reviewText']).sentiment
        if user not in users:
            users[user] = new_user()
        users[user]['text'] += [review['reviewText']]
        users[user]['polarity'] += [polarity]
        users[user]['subjectivity'] += [subjectivity]
        users[user]['rating'] += [float(review['overall'])]
        users[user]['item'] += [item]
        users[user]['helpful_vote'] += [int(review['helpful']['nHelpful'])]
        users[user]['total_vote'] += [int(review['helpful']['outOf'])]
        for category in categories:
            if category not in users[user]['category']:
                users[user]['category'][category] = 0
            users[user]['category'][category] += 1
        items[item]['review_num'] += 1
        items[item]['avg_polarity'] += [polarity]
        items[item]['avg_subjectivity'] += [subjectivity]
        items[item]['avg_rating'] += [float(review['overall'])]
            
    for user in users:
        avg_rating = sum(users[user]['rating'])/float(len(users[user]['rating']))
        avg_polarity = sum(users[user]['polarity'])/float(len(users[user]['polarity']))
        avg_subj = sum(users[user]['subjectivity'])/float(len(users[user]['subjectivity']))
        purchase_num = len(users[user]['item'])
        distinct_cats = len(users[user]['category'])
        
        running_sum = 0
        count = 0
        for asin in users[user]['item']:
            if items[asin]['price'] != None:
                running_sum += items[asin]['price']
                count += 1 
        try:
            avg_price = running_sum / float(count)
        except:
            avg_price = None
        
        users[user]['stats']['avg_rating'] = avg_rating
        users[user]['stats']['avg_polarity'] = avg_polarity
        users[user]['stats']['avg_subjectivity'] = avg_subj
        users[user]['stats']['purchase_num'] = purchase_num
        users[user]['stats']['distinct_cats'] = distinct_cats
        users[user]['stats']['avg_price'] = avg_price
        
    for item in items:
        try:
            items[item]['avg_polarity'] = sum(items[item]['avg_polarity'])/float(len(items[item]['avg_polarity']))
            items[item]['avg_subjectivity'] = sum(items[item]['avg_subjectivity'])/float(len(items[item]['avg_subjectivity']))
            items[item]['avg_rating'] = sum(items[item]['avg_rating'])/float(len(items[item]['avg_rating']))
        except:
            items[item]['avg_polarity'] = None
            items[item]['avg_subjectivity'] = None
            items[item]['avg_rating'] = None
        
    return users
 
items = build_item_data()
users = build_user_data(items)
pickle.dump(items, open('items.p', 'wb'))
pickle.dump(users, open('users.p', 'wb'))

In [94]:
# load things from last cell from save file
# uncomment after running first cell and comment out last 4 lines in first cell

import pickle

#items = pickle.load(open('items.p', 'rb'))
#users = pickle.load(open('users.p', 'rb')) # 1 gig large, beware

In [95]:
# Extra calculations needed later

global_mean = sum([users[user]['stats']['avg_rating'] * \
                   users[user]['stats']['purchase_num'] for user in users]) / \
              float(sum([users[user]['stats']['purchase_num'] for user in users]))
purchase_num_mean = sum([users[user]['stats']['purchase_num'] for user in users]) / \
                    float(len(users))
helpful_mean = sum([sum(users[user]['helpful_vote']) for user in users]) / \
               float(sum([sum(users[user]['total_vote']) for user in users]))
    
for user in users:
    users[user]['stats']['avg_helpful'] = sum(users[user]['helpful_vote']) / float(sum(users[user]['total_vote']))

In [96]:
# Rating Prediction: Part 1

def all_info_rating(user, item):
    u_avg = users[user]['stats']['avg_rating']
    i_avg = items[item]['avg_rating']
    try:
        return (u_avg + i_avg + global_mean) / float(3)
    except:
        return only_user(user)

def only_item_rating(item):
    i_avg = items[item]['avg_rating']
    if i_avg == None:
        return global_mean
    return (i_avg + global_mean) / float(2)

def run_file_rating(users, items):
    fname = 'amazon_reviews_Electronics/pairs_Rating.txt'
    data = open(fname, 'r').read().split('\n')
    for i in range(len(data)):
        data[i] = data[i].split('-')
    while True:
        if data[-1] != ['']:
            break
        data = data[:-1]
    for i in range(1, len(data)):
        user = data[i][0]
        item = data[i][1]
        if user in users and item in items:
            data[i] += [all_info_rating(user, item)]
        else:
            data[i] += [only_item_rating(item)]
    s = 'reviewerID-asin,prediction\n'
    for i in range(1, len(data)):
        s += data[i][0] + '-' + data[i][1] + ',' + str(data[i][2]) + '\n'
    open('rating_prediction.csv', 'w').write(s)
    print 'Complete!'
    
run_file_rating(users, items)

Complete!


In [97]:
# Purchase Prediction: Part 2

def all_info_purchase(user, item):
    return int(items[item]['review_num'] > purchase_num_mean)

def only_item_purchase(item):
    return int(items[item]['review_num'] > purchase_num_mean)

def run_file_purchase(users, items):
    fname = 'amazon_reviews_Electronics/pairs_Purchase.txt'
    data = open(fname, 'r').read().split('\n')
    for i in range(len(data)):
        data[i] = data[i].split('-')
    while True:
        if data[-1] != ['']:
            break
        data = data[:-1]
    for i in range(1, len(data)):
        user = data[i][0]
        item = data[i][1]
        if user in users and item in items:
            data[i] += [all_info_purchase(user, item)]
        else:
            data[i] += [only_item_purchase(item)]
    s = 'reviewerID-asin,prediction\n'
    for i in range(1, len(data)):
        s += data[i][0] + '-' + data[i][1] + ',' + str(data[i][2]) + '\n'
    open('purchase_prediction.csv', 'w').write(s)
    print 'Complete!'
    
run_file_purchase(users, items)

Complete!


In [98]:
# Helpfulness Prediction: Part 3

def run_file_helpful(users, items):
    helpful = 'amazon_reviews_Electronics/helpful.json'
    item = None
    data = []
    for line in open(helpful):
        item = eval(line)
        data += [[item['reviewerID'], 
                  item['asin'], 
                  item['outOf'], 
                  round(((item['outOf']*helpful_mean) + \
                         (item['outOf']*users[user]['stats']['avg_helpful'])) / float(2))]]
    s = 'reviewerID-asin-outOf,prediction\n'
    for i in range(len(data)):
        s += data[i][0] + '-' + data[i][1] + '-' + str(data[i][2]) + ',' + str(data[i][3]) + '\n'
    open('helpful_prediction.csv', 'w').write(s)
    print 'Complete!'
    
run_file_helpful(users, items)

Complete!


WRITTEN REPORT


(in general): As you can see in my first code cell, I gathered and aggregated a lot more data than I ended up actually needing. After looking into the data, different types of averages worked best. I did not end up using any NLP results, along with category or brand.

Task 1 - For task one, I implemented an aggregation of global, specific user and specific item averages. I chose this to model the user and item bias on the global mean, where I gave equal weight to the global mean rating, the user average ratings and the item average ratings. It turned out to work quite well on the unknown testing data.

Task 2 - For task two, I only used one piece of data: whether the number of times this item was purchased was greater than the mean of purchased items. I tried many other things and nothing could beat it. In my other attempts, I used the user's propensity to buy anything at all, buy things at certain prices, or buy things with certain average ratings. Nothing outperformed the simple check I ended up using. 

Task 3 - For task three, I used a similiar method as with task one, giving equal weight to two averages, the global percentage of reviews that are helpful, and how helpful each users' reviews are. Again, I tried several other methods and this performed the best.

In [ ]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()